<a href="https://colab.research.google.com/github/sanbgos/Se-alesysistemas/blob/main/Parcial_2_SyS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 2 - Señales y Sistemas 2025-I
**Nombre:** [Santiago Burgos Salazar]  
**Profesor:** Andrés Marino Álvarez Meza, Ph.D.  
**Universidad Nacional de Colombia - Sede Manizales**

## Punto 1. Modelado y simulación del sistema masa-resorte-amortiguador y su circuito equivalente

Este cuaderno contiene el desarrollo del modelo dinámico, simulaciones y visualización para comparar el comportamiento del sistema mecánico y su análogo eléctrico.


In [ ]:
!pip install streamlit control matplotlib -q
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared
!mv cloudflared /usr/local/bin/cloudflared
!pip install yt-dlp -q
!apt-get install ffmpeg -y

#**Pagina principal**

In [ ]:
%%writefile 0_Inicio.py
import streamlit as st

# --- Configuración de la página ---
st.set_page_config(
    page_title="Parcial 2 - Señales y Sistemas",
    page_icon="📘",
    layout="centered"
)

# --- Título principal ---
st.markdown("<h1 style='text-align: center; color: #004488;'>📘 Bienvenido al Parcial 2 de Señales y Sistemas</h1>", unsafe_allow_html=True)
st.markdown("---")

# --- Introducción ---
st.markdown("""
Esta aplicación interactiva fue desarrollada como entrega del **Parcial 2** del curso *Señales y Sistemas (2025-1)*.

Contiene el desarrollo completo, análisis y simulaciones de los siguientes puntos:

---

### 📌 Contenido del Parcial

#### 1️⃣ Sistema Masa-Resorte-Amortiguador y su Circuito Eléctrico Equivalente

- Obtención de la ecuación diferencial del sistema mecánico
- Derivación de la función de transferencia
- Estimación de parámetros físicos y eléctricos
- Simulación de la respuesta al impulso, escalón y rampa
- Análisis de Bode y plano de polos y ceros

#### 2️⃣ Modulación y Demodulación SSB-AM (Single Side Band Amplitude Modulation)

- Modelado matemático del proceso de modulación y demodulación SSB-AM
- Análisis en el dominio del tiempo y frecuencia (con Fourier)
- Visualización del proceso con una señal de entrada tipo:
  - i) Pulso rectangular
  - ii) Fragmento de audio (5 segundos de una canción)
- Implementación de filtros digitales IIR requeridos
- Diagrama de Bode y plano de polos y ceros de los filtros
- Visualización de señales en cada etapa del sistema (modulación, filtrado, demodulación)

---

### 👤 Información del Estudiante

- **Nombre:** Santiago Burgos Salazar
- **Cédula:** 1121821274
- **Curso:** Señales y Sistemas – 2025-1
- **Profesor:** Andrés Marino Álvarez Meza, Ph.D.

---

### 🧭 ¿Cómo navegar?

Utiliza el **menú lateral izquierdo (👈)** para acceder a cada punto del parcial.
Cada sección está diseñada para explorar de forma interactiva conceptos clave del curso, combinando visualización en tiempo real, transformadas, filtros y respuestas del sistema.
""")

# --- Barra lateral ---
st.sidebar.success("Selecciona un punto del parcial para iniciar.")


#**Respuesta punto 1**

In [ ]:
%%writefile pages/1_Punto_1.py
import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

st.set_page_config(page_title="Punto 1 - Sistema Segundo Orden", page_icon="⚙️", layout="wide")

# --- Título ---
st.markdown("<h1 style='text-align: center; color: #004488;'>⚙️ Punto 1 - Sistema Masa-Resorte-Amortiguador y su Equivalente Eléctrico</h1>", unsafe_allow_html=True)
st.markdown("---")

# --- Introducción ---
st.markdown("""
Este simulador interactivo analiza un sistema de segundo orden desde dos perspectivas:

- 🧮 Modelo Mecánico: masa, resorte y amortiguador.
- 🔌 Modelo Eléctrico Equivalente: circuito RLC serie-paralelo.

Permite explorar cómo el amortiguamiento y la frecuencia natural afectan las respuestas temporales y frecuenciales.
""")

# --- Controles ---
st.sidebar.header("⚙️ Parámetros del Sistema")

tipo = st.sidebar.selectbox("Tipo de respuesta", ["Subamortiguada", "Crítica", "Sobreamortiguada", "Inestable"])

if tipo == "Subamortiguada":
    zeta = st.sidebar.slider("Amortiguamiento ζ", 0.01, 0.99, 0.3, 0.01)
elif tipo == "Crítica":
    zeta = 1.0
    st.sidebar.info("ζ = 1 para amortiguamiento crítico.")
elif tipo == "Sobreamortiguada":
    zeta = st.sidebar.slider("Amortiguamiento ζ", 1.1, 5.0, 1.5, 0.1)
else:
    zeta = st.sidebar.slider("Amortiguamiento ζ", -1.0, -0.01, -0.5, 0.01)

omega_n = st.sidebar.slider("Frecuencia natural ωn (rad/s)", 1.0, 20.0, 5.0, 0.5)

# --- Funciones de transferencia ---
num = [omega_n**2]
den_ol = [1, 2*zeta*omega_n, omega_n**2]
den_cl = [1, 2*zeta*omega_n, omega_n**2 + omega_n**2]  # Lazo cerrado con realimentación unitaria
sys_ol = signal.TransferFunction(num, den_ol)
sys_cl = signal.TransferFunction(num, den_cl)

# --- Parámetros físicos ---
m = 1.0
k = omega_n**2 * m
c = 2*zeta*omega_n * m

C = 1
L = 1 / (omega_n**2 * C)
R = 1 / (2*zeta*omega_n*C) if zeta != 0 else float('inf')

# --- Mostrar parámetros ---
st.markdown("---")
st.markdown("### 📐 Parámetros del Sistema")

col1, col2 = st.columns(2)

with col1:
    st.markdown("#### 🧮 Sistema Mecánico")
    st.code(f"""Masa (m):      {m:.2f} kg
Constante del resorte (k): {k:.2f} N/m
Amortiguamiento (c):       {c:.2f} Ns/m""")

with col2:
    st.markdown("#### 🔌 Sistema Eléctrico Equivalente")
    st.code(f"""Inductancia (L): {L:.4f} H
Resistencia (R):   {R:.4f} Ω
Capacitancia (C):  {C:.1f} F""")

# --- Análisis ---
st.markdown("---")
st.markdown("### 📊 Análisis del Comportamiento del Sistema")

tabs = st.tabs(["📈 Bode", "🌀 Polos y Ceros", "⚡ Impulso", "⬆️ Escalón", "⤴️ Rampa"])

# Tiempo común
t = np.linspace(0, 5, 1000)

# Tab 1: Bode
with tabs[0]:
    st.subheader("📈 Diagrama de Bode")
    w = np.logspace(-1, 2, 500)
    w_ol, mag_ol, phase_ol = signal.bode(sys_ol, w=w)
    w_cl, mag_cl, phase_cl = signal.bode(sys_cl, w=w)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6))
    ax1.semilogx(w_ol, mag_ol, label="Lazo Abierto")
    ax1.semilogx(w_cl, mag_cl, label="Lazo Cerrado", linestyle='--')
    ax1.set_ylabel("Magnitud (dB)")
    ax1.grid(True)
    ax1.legend()

    ax2.semilogx(w_ol, phase_ol, label="Lazo Abierto")
    ax2.semilogx(w_cl, phase_cl, label="Lazo Cerrado", linestyle='--')
    ax2.set_ylabel("Fase (°)")
    ax2.set_xlabel("Frecuencia (rad/s)")
    ax2.grid(True)
    ax2.legend()

    st.pyplot(fig)

# Tab 2: Polos y ceros
with tabs[1]:
    st.subheader("🌀 Diagrama de Polos y Ceros")
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(np.real(sys_ol.poles), np.imag(sys_ol.poles), marker='x', color='blue', label='Polos LA')
    ax.scatter(np.real(sys_cl.poles), np.imag(sys_cl.poles), marker='x', color='red', label='Polos LC')
    ax.axhline(0, color='black', lw=0.5)
    ax.axvline(0, color='black', lw=0.5)
    ax.set_xlabel("Re")
    ax.set_ylabel("Im")
    ax.grid(True)
    ax.legend()
    st.pyplot(fig)

# Tab 3: Impulso
with tabs[2]:
    st.subheader("⚡ Respuesta al Impulso")
    _, y_ol = signal.impulse(sys_ol, T=t)
    _, y_cl = signal.impulse(sys_cl, T=t)
    fig, ax = plt.subplots()
    ax.plot(t, y_ol, label="Lazo Abierto")
    ax.plot(t, y_cl, linestyle="--", label="Lazo Cerrado")
    ax.set_xlabel("Tiempo (s)")
    ax.set_ylabel("Amplitud")
    ax.grid(True)
    ax.legend()
    st.pyplot(fig)

# Tab 4: Escalón
with tabs[3]:
    st.subheader("⬆️ Respuesta al Escalón")
    _, y_ol = signal.step(sys_ol, T=t)
    _, y_cl = signal.step(sys_cl, T=t)
    fig, ax = plt.subplots()
    ax.plot(t, y_ol, label="Lazo Abierto")
    ax.plot(t, y_cl, linestyle="--", label="Lazo Cerrado")
    ax.set_xlabel("Tiempo (s)")
    ax.set_ylabel("Amplitud")
    ax.grid(True)
    ax.legend()
    st.pyplot(fig)

# Tab 5: Rampa
with tabs[4]:
    st.subheader("⤴️ Respuesta a la Rampa")
    sys_ramp_ol = signal.TransferFunction(num, np.polymul(den_ol, [1, 0]))
    sys_ramp_cl = signal.TransferFunction(num, np.polymul(den_cl, [1, 0]))
    _, y_ol = signal.step(sys_ramp_ol, T=t)
    _, y_cl = signal.step(sys_ramp_cl, T=t)
    fig, ax = plt.subplots()
    ax.plot(t, t, 'k:', label="Entrada Rampa")
    ax.plot(t, y_ol, label="Salida Lazo Abierto")
    ax.plot(t, y_cl, linestyle="--", label="Salida Lazo Cerrado")
    ax.set_xlabel("Tiempo (s)")
    ax.set_ylabel("Amplitud")
    ax.grid(True)
    ax.legend()
    st.pyplot(fig)


# 📡 Punto 2 - Modulación y Demodulación SSB-AM

En este punto se aborda el análisis y simulación del proceso de **modulación y demodulación por amplitud en banda lateral única (SSB-AM)**, una técnica eficiente utilizada en sistemas de telecomunicaciones para transmitir señales con menor ancho de banda y potencia.

El objetivo es construir un modelo matemático del proceso de **modulación SSB-AM** y su respectiva **demodulación**, expresado tanto en el **dominio del tiempo** como en el **dominio de la frecuencia** mediante el uso de la **Transformada de Fourier**.


In [ ]:
%%writefile pages/2_Punto_2.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, sosfilt, sosfreqz, tf2zpk, hilbert
from scipy.fft import fft, fftfreq, fftshift
from scipy.io import wavfile
import os
import subprocess
import yt_dlp

# Configuración general de la página
st.set_page_config(page_title="Punto 2 - Modulación SSB-AM", page_icon="📡", layout="wide")
st.markdown("<h1 style='text-align: center; color: #004488;'>📡 Punto 2 - Modulación y Demodulación SSB-AM</h1>", unsafe_allow_html=True)
st.markdown("---")

# -----------------------------------
# FUNCIONES AUXILIARES
# -----------------------------------
def extraer_polos_y_zeros_validos(sos):
    zeros, poles = [], []
    for section in sos:
        b, a = section[:3], section[3:]
        if np.allclose(a, 0):
            continue
        try:
            z, p, _ = tf2zpk(b, a)
            zeros.extend(z)
            poles.extend(p)
        except Exception:
            continue
    return np.array(zeros), np.array(poles)

def plot_bode_y_pz(sos, fs):
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))

    # Bode
    w, h = sosfreqz(sos, worN=2000, fs=fs)
    axs[0].plot(w, 20 * np.log10(np.abs(h)))
    axs[0].set_title("Diagrama de Bode")
    axs[0].set_xlabel("Frecuencia (Hz)")
    axs[0].set_ylabel("Magnitud (dB)")
    axs[0].grid(True)

    # Polos y ceros
    z, p = extraer_polos_y_zeros_validos(sos)
    axs[1].scatter(np.real(z), np.imag(z), marker='o', facecolors='none', edgecolors='b', label='Ceros')
    axs[1].scatter(np.real(p), np.imag(p), marker='x', color='r', label='Polos')
    axs[1].axhline(0, color='black', lw=0.5)
    axs[1].axvline(0, color='black', lw=0.5)
    axs[1].set_title("Plano de Polos y Ceros")
    axs[1].grid(True)
    axs[1].legend()

    return fig

def load_audio_from_youtube(url, duration_s=5, start_s=0):
    temp_wav_file = 'temp_audio.wav'
    cropped_wav_file = 'cropped_audio.wav'

    try:
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'wav'}],
            'outtmpl': 'temp_audio',
            'quiet': True,
        }
        if os.path.exists(temp_wav_file):
            os.remove(temp_wav_file)
        if os.path.exists(cropped_wav_file):
            os.remove(cropped_wav_file)

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        command = [
            'ffmpeg', '-i', temp_wav_file, '-ss', str(start_s),
            '-t', str(duration_s), '-acodec', 'pcm_s16le',
            '-ar', '44100', '-ac', '1', cropped_wav_file, '-y'
        ]
        subprocess.run(command, check=True, capture_output=True)

        fs_audio, audio_data = wavfile.read(cropped_wav_file)
        if audio_data.ndim > 1:
            audio_data = audio_data.mean(axis=1)
        audio_data = audio_data / np.max(np.abs(audio_data))
        m_t = audio_data
        t = np.linspace(start_s, start_s + len(m_t) / fs_audio, len(m_t), endpoint=False)

        return m_t, fs_audio, t
    except Exception as e:
        st.error(f"❌ Error al procesar el audio de YouTube: {e}")
        return None, None, None

def ssb_modulate(m_t, t, fs, fc, tipo='USB'):
    m_hilbert = np.imag(hilbert(m_t))
    if tipo == 'USB':
        s_t = m_t * np.cos(2 * np.pi * fc * t) - m_hilbert * np.sin(2 * np.pi * fc * t)
    else:
        s_t = m_t * np.cos(2 * np.pi * fc * t) + m_hilbert * np.sin(2 * np.pi * fc * t)
    return s_t

def ssb_demodulate(s_t, t, fs, fc, sos_pb):
    demod = s_t * 2 * np.cos(2 * np.pi * fc * t)
    return sosfilt(sos_pb, demod)

def plot_time(t, sig, title):
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.plot(t, sig)
    ax.set_title(title)
    ax.set_xlabel("Tiempo (s)")
    ax.set_ylabel("Amplitud")
    ax.grid(True)
    return fig

def plot_freq(sig, fs, title, xlim=None):
    N = len(sig)
    f = fftfreq(N, 1 / fs)
    Y = np.abs(fft(sig))
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.plot(fftshift(f), fftshift(Y))
    ax.set_title(title)
    ax.set_xlabel("Frecuencia (Hz)")
    ax.set_ylabel("Magnitud")
    if xlim: ax.set_xlim(xlim)
    ax.grid(True)
    return fig

# -----------------------------------
# INTERFAZ STREAMLIT
# -----------------------------------
st.sidebar.title("⚙ Parámetros de la Simulación")
tipo_mensaje = st.sidebar.selectbox("Tipo de señal mensaje", ["Pulso Rectangular", "Audio de YouTube"])
fc = st.sidebar.slider("Frecuencia de portadora (Hz)", 1000, 20000, 10000, step=500)
lado = st.sidebar.radio("Seleccione la banda lateral", ["USB", "LSB"])
duracion = st.sidebar.slider("Duración (s)", 1, 10, 5)
fs = 44100  # Frecuencia de muestreo

if tipo_mensaje == "Pulso Rectangular":
    t = np.linspace(0, duracion, duracion * fs, endpoint=False)
    ancho_pulso = st.sidebar.slider("Ancho del pulso (s)", 0.1, 1.0, 0.5)
    m_t = np.zeros_like(t)
    m_t[t < ancho_pulso] = 1
else:
    url = st.sidebar.text_input("URL de YouTube", "https://www.youtube.com/watch?v=QH2-TGUlwu4")
    inicio = st.sidebar.slider("Inicio (s)", 0, 60, 5)
    m_t, fs, t = load_audio_from_youtube(url, duration_s=duracion, start_s=inicio)

# -----------------------------------
# PROCESAMIENTO Y GRAFICACIÓN
# -----------------------------------
if m_t is not None:
    st.subheader("🔊 Señal de Mensaje")
    st.pyplot(plot_time(t, m_t, "Mensaje en el tiempo"))
    st.pyplot(plot_freq(m_t, fs, "Mensaje en frecuencia", xlim=(-10000, 10000)))
    if tipo_mensaje == "Audio de YouTube":
        st.audio(m_t, format="audio/wav", sample_rate=fs)

    st.subheader("📡 Señal Modulada SSB")
    s_t = ssb_modulate(m_t, t, fs, fc, tipo=lado)
    st.pyplot(plot_time(t, s_t, "Señal SSB en el tiempo"))
    st.pyplot(plot_freq(s_t, fs, "Señal SSB en frecuencia", xlim=(-20000, 20000)))

    st.subheader("🔁 Señal Demodulada")
    sos_pb = butter(10, 4000, fs=fs, output='sos')
    m_rec = ssb_demodulate(s_t, t, fs, fc, sos_pb)
    st.pyplot(plot_time(t, m_rec, "Señal Recuperada en el tiempo"))
    st.pyplot(plot_freq(m_rec, fs, "Señal Recuperada en frecuencia", xlim=(-10000, 10000)))
    if tipo_mensaje == "Audio de YouTube":
        st.audio(m_rec, format="audio/wav", sample_rate=fs)

    st.subheader("🧭 Análisis del Filtro IIR Pasa Bajas")
    st.pyplot(plot_bode_y_pz(sos_pb, fs))
else:
    st.warning("⚠️ Espera mientras se carga la señal de mensaje.")


In [ ]:
# Lanzar Streamlit y Cloudflare Tunnel
!streamlit run 0_Inicio.py &>/content/logs.txt &
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

# Esperar y mostrar URL pública
import time, re
time.sleep(12)  # Más tiempo por seguridad

try:
    with open('/content/cloudflared.log') as f:
        log_data = f.read()
        match = re.search(r"https://[-\w.]+\.trycloudflare\.com", log_data)
        if match:
            print("✅ Tu app está disponible en:", match.group(0))
        else:
            print("❌ No se encontró el enlace. Verifica el archivo log.")
except FileNotFoundError:
    print("❌ No se pudo encontrar el archivo de log.")
